# KICP ADR CMB SUMMER SCHOOL NOTEBOOK (RBT/AT/NK)

## 1. Initialization 
*(will be different depending on your cryostat and software)*

In [1]:
# Various imports
import serial, threading, sys, time, os, datetime, warnings, numpy as np, matplotlib.pyplot as plt
from IPython.display import display, clear_output
from ipywidgets import FloatProgress, HTML, VBox
from timeit import default_timer as timer

# We will be using our homemade library to talk to ADR via serial
sys.path.append('/home/pi/Desktop/ADRNX/SRS')
import SRS
import Switchduino as sd

# Initialize mainframe controller (with config file, suppressing debug statements)
mf = SRS.SIM900.SIM900('/home/pi/Desktop/ADRNX/SRS/config.yaml',debug=False)
mf.init_mainframe()

# Create module objects (coresponding to those in the rack) and add them to mainframe
s921 = SRS.SIM921.SIM921(mf,name='s921',debug=False)
s922 = SRS.SIM922.SIM922(mf,name='s922',debug=False)
s960 = SRS.SIM960.SIM960(mf,name='s960',debug=False)
s970 = SRS.SIM970.SIM970(mf,name='s970',debug=False)
mf.register_and_init_mods([s921, s922, s960, s970])

# Finally, start communication threads (this will enable all the backend virtual queues and messaging)
mf.start_comm_threads()

In [2]:
def make_bar(minv=0,maxv=1,prefix='Voltage: ',postfix='/2.0V',style='info'):
    progress = FloatProgress(min=minv,max=maxv,step=(maxv-minv)/1000)
    progress.bar_style = style
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)
    
    label.value = prefix+''+postfix
    return [progress, label, box,prefix,postfix]
    
def update_bar(bar,val,fmt='{:.3f}'):
    bar[0].value = val
    bar[1].value = bar[3]+fmt.format(val)+bar[4]
    if val ==  bar[0].max:  
        bar[0].bar_style = 'success'  
        
def check_for_exceptions():
    try:
        print(s960.excq.get_nowait())
        raise RuntimeError("Got s960 exc")
    except:
        pass
    try:
        print(s921.excq.get_nowait())
        raise RuntimeError("Got s921 exc")
    except:
        pass

## 2a. Preparation part 1
To make sure you don't quench magnet and kill your PhD career, it is prudent to do a lot of checks before cycling. Your KEPCO voltage follower power supply output should be 0, since we are playing with sense resistors here. To begin, cycle the heat switch (to relieve any mechanical stresses).

In [3]:
sd.heatswitch_open()
sd.heatswitch_close()

Heat Switch opening
4K-GGG LED OFF
4K-FAA LED OFF
GGG-FAA LED OFF
Heat Switch is opened
Heat Switch closing
4K-GGG LED ON
4K-FAA LED ON
GGG-FAA LED ON
Heat Switch is closed


Then make sure your relay is in the right mode (1 Ohm resistor aka magcycle mode)

In [4]:
if abs(s960.get_output_voltage())<0.01: sd.relay_switch_magcycle()
else: raise Exception()

## 2b. Preparation part 2
Now do a bunch of checks on your voltage controller

In [5]:
# Check actual controller output - it might not be 0 for legitimate reasons, so just warn
if abs(s960.get_output_voltage())>0.01: warnings.warn("Current is not zero - BE CAREFUL")

# If we are in manual mode, warn if current output is not zero, otherwise error out (we should not be PIDing atm)
if s960.get_PID_mode() == 0:
    if abs(s960.get_manual_output())>0.01: warnings.warn("Manual output is not zero - BE CAREFUL")
else:
    raise RuntimeError("You should exit PID mode before running this!")
    
# These are not as important
if s960.get_offset_onoff() != 0:
    if s960.get_offset != 0.0: raise RuntimeError("Running with offsets screws things up - who enabled them?")
if s960.get_ramp_state() != 0: raise RuntimeError("You are ramping...that should not happen...like ever!")  
    
# Set up things the way we want, some are redundant...just in case of random undergrads and other acts of god
s960.set_PID_mode(0); assert s960.get_PID_mode() == 0
s960.set_offset(0); assert s960.get_offset() == 0
s960.set_offset_onoff(0); assert s960.get_offset_onoff() == 0

s960.set_setpoint(0); assert s960.get_setpoint() == 0
s960.set_setpoint_mode(0); assert s960.get_setpoint_mode() == 0
s960.set_ramp_onoff(0); assert s960.get_ramp_onoff() == 0

In [6]:
# Setting limits both in SIM960 and also in code to prevent excess voltages
ul = 5.5
s960.set_upper_lim(ul-0.1); assert s960.get_upper_lim() == ul-0.1
s960.manlim_high = ul

ll = -0.1
s960.set_lower_lim(-0.1); assert s960.get_lower_lim() == -0.1
s960.manlim_low = -0.1

In [7]:
#Finally, let's see what the settings are
s960.tellastory()

My name is s960 (ID: Stanford_Research_Systems,SIM960,s/n014669,ver2.17)
Status is  0b10000
 Settings:
  PID mode is       0 (MAN 0, PID 1)
  Setpt source is   0 (INT 0, EXT 1)
  RAMP mode is      0 (OFF 0, ON 1)
  RAMP rate is      0.01 (V/s)
  RAMP status is    0 (IDLE 0, PENDING 1, RAMPING 2, PAUSED 3)
 PID:
  P term is         +2.00E+01  [state: 1 (OFF 0, ON 1)]
  I term is         +2.00E-01  [state: 1 (OFF 0, ON 1)]
  D term is         +1.00E-05  [state: 1 (OFF 0, ON 1)]
 Setpoints:
  Manual output is  0.0 (V)
  PID setpoint is   0.0 (V)
  Offset is         0.0 (V)    [state: 0 (OFF 0, ON 1)]
  Limits are        -0.1<->5.4 (V)
 Voltages:
  Output V is       +0.000781 
  Measure V is      +2.373784 
  Setpoint V is     -0.000087 


**If you are happy about current settings, proceed to cycling (if outputs are 0, you can now turn on KEPCO if not already on)**

## 3. Mag up
*First cycle stage is ramping up current while keeping back emf sufficiently low. Final target depends on your magnet spec (~9A max in our case), as well as the desired temperature/hold time. There are complicated tradeoffs - while this part runs, take a look at HPD manual for some interesting curves.*

In [8]:
target = 8.2 #A
maxemf = 0.12#V
vstep = 0.005 #Max slew rate in V/s
target=target*0.55 #convert to V

In [9]:
# Define some empty lists for logging
emf_series, vmeas_series, times_emf, times_vmeas = ([] for i in range(4))
tempFAA_series, times_tempFAA = ([] for i in range(2))

# Get current output twice - if this is somehow wrong, things will go very bad
vout = s960.get_manual_output()
vout2 = s960.get_manual_output()
assert vout == vout2

# Can go either direction
if vout > target: 
    direction = 0 #down
elif vout == target: 
    warnings.warn("Target same as current value")
    direction = 1
else:
    direction = 1 #up
delta = abs(vout - target)

# Flush communications
mf.clear_virtual_queues()
mf.clear_hw_queues()

# Get some pretty bars
barv = make_bar(min(target,vout),max(target,vout),prefix='Measured output: ',postfix='V',style='warning')
baremf = make_bar(0,300,prefix='Back EMF: ',postfix='mV',style='danger')
bart = make_bar(0,3.5,prefix='FAA: ',postfix='K',style='info')

print("Starting run from {} to {} (dir:{}) at {} with emf limit of {}".format(vout,target,direction,vstep,maxemf))
start = timer()
try:
    # Start streaming EMF channel data
    s970.start_data_stream(SRS.SIM970.CHANNELS.MAGEMF)
    # And also temperatures
    s921.start_data_stream()
    print("Entering main loop")
    while(abs(vout - target) > 0.004):
        loop_start = timer()        
        # Loop until backemf is low enough, but at least once (poor mans do-while loop)
        while True:            
            gotevent = s970.await_next_event(tm=1.0,clear_before=True,clear_after=True)
            if not gotevent:
                raise RuntimeError("Did not get voltage in time - aborting")   
            emf = abs(s970.lastvalues[SRS.SIM970.CHANNELS.MAGEMF])
            emf_series.append(emf)
            times_emf.append(timer()-start)
            update_bar(baremf,emf*1000)            
            if abs(emf) < abs(maxemf): break
            
        # Set new manual voltage
        if (direction):
            if (vout + vstep < target): vnew = vout + vstep               
            else: vnew = target                   
        else:
            if (vout - vstep > target): vnew = vout - vstep                
            else: vnew = target                 
        s960.set_manual_output(vnew)
        time.sleep(0.2)
        
        # Check that new manual voltage is as intended (setting and actual output):
        try:
            vout_chk = s960.get_manual_output()
            time.sleep(0.2)
            vmeas_chk = s960.get_output_voltage()
            time.sleep(0.2)
        except Exception as e: 
            raise RuntimeError("Could not get s960 settings - ABORTING")    
                
        # Grab new temperature reading        
        if s921.await_next_event(tm=1.1,clear_before=False,clear_after=True):
            temp = s921.lastvalues[1]
            tempFAA_series.append(temp)
            times_tempFAA.append(s921.lastdatatimes[1]-start)
            update_bar(bart,temp)
        else:
            print("Got not temp data...weird...")
            temp = 0
            
        print("{:05.2f}% | VMEAS: {:.5f} | VRESP: {:.5f} | VOLD: {:.5f} | VNEW:{:.5f} | EMF: {:.6f} | TEMP: {:.6f}".format(\
            100-abs(vout - target)*100/delta,vmeas_chk,vout_chk,vout,vnew,emf,temp))
        vout = vnew; vmeas_series.append(vmeas_chk); times_vmeas.append(timer()-start)
        update_bar(barv,vout_chk)
        
        # Error checking
        check_for_exceptions()
        assert abs(vout_chk - vnew) < 0.000001 #rounding of floats a problem 
        assert abs(vmeas_chk - vnew) < 0.02        
        
        # Wait remainder of 1 second
        while(timer()-loop_start < 1.0): time.sleep(0.05)  
    print("MAGCYCLE DONE - took {:2f} minutes".format((timer()-start)/60))
except Exception as e:
    print(e)
finally:
    # Stop data and clean up
    s970.stop_data_stream()
    s921.stop_data_stream()
    time.sleep(0.2)
    mf.clear_virtual_queues()

Starting run from 0.0 to 4.51 (dir:1) at 0.005 with emf limit of 0.12
Entering main loop
00.00% | VMEAS: 0.00081 | VRESP: 0.00500 | VOLD: 0.00000 | VNEW:0.00500 | EMF: 0.000391 | TEMP: 2.844262
00.11% | VMEAS: 0.00931 | VRESP: 0.01000 | VOLD: 0.00500 | VNEW:0.01000 | EMF: 0.009971 | TEMP: 2.850548
00.22% | VMEAS: 0.01579 | VRESP: 0.01500 | VOLD: 0.01000 | VNEW:0.01500 | EMF: 0.019237 | TEMP: 2.841948
00.33% | VMEAS: 0.02075 | VRESP: 0.02000 | VOLD: 0.01500 | VNEW:0.02000 | EMF: 0.028185 | TEMP: 2.841334
00.44% | VMEAS: 0.02078 | VRESP: 0.02500 | VOLD: 0.02000 | VNEW:0.02500 | EMF: 0.036773 | TEMP: 2.852243
00.55% | VMEAS: 0.02577 | VRESP: 0.03000 | VOLD: 0.02500 | VNEW:0.03000 | EMF: 0.045002 | TEMP: 2.849346
00.67% | VMEAS: 0.03546 | VRESP: 0.03500 | VOLD: 0.03000 | VNEW:0.03500 | EMF: 0.052348 | TEMP: 2.847602
00.78% | VMEAS: 0.04046 | VRESP: 0.04000 | VOLD: 0.03500 | VNEW:0.04000 | EMF: 0.060051 | TEMP: 2.845632
00.89% | VMEAS: 0.04547 | VRESP: 0.04500 | VOLD: 0.04000 | VNEW:0.04500

08.54% | VMEAS: 0.39083 | VRESP: 0.39000 | VOLD: 0.38500 | VNEW:0.39000 | EMF: 0.119654 | TEMP: 2.889832
08.65% | VMEAS: 0.39584 | VRESP: 0.39500 | VOLD: 0.39000 | VNEW:0.39500 | EMF: 0.119383 | TEMP: 2.898875
08.76% | VMEAS: 0.40058 | VRESP: 0.40000 | VOLD: 0.39500 | VNEW:0.40000 | EMF: 0.119109 | TEMP: 2.919591
08.87% | VMEAS: 0.40562 | VRESP: 0.40500 | VOLD: 0.40000 | VNEW:0.40500 | EMF: 0.119243 | TEMP: 2.914195
08.98% | VMEAS: 0.41064 | VRESP: 0.41000 | VOLD: 0.40500 | VNEW:0.41000 | EMF: 0.119998 | TEMP: 2.911412
09.09% | VMEAS: 0.41062 | VRESP: 0.41500 | VOLD: 0.41000 | VNEW:0.41500 | EMF: 0.119652 | TEMP: 2.923079
09.20% | VMEAS: 0.42072 | VRESP: 0.42000 | VOLD: 0.41500 | VNEW:0.42000 | EMF: 0.119373 | TEMP: 2.919900
s960 | Recovered: |'+00.420724\r'|, buf |+00.|
09.31% | VMEAS: 0.42072 | VRESP: 0.42500 | VOLD: 0.42000 | VNEW:0.42500 | EMF: 0.119135 | TEMP: 2.922714
09.42% | VMEAS: 0.42575 | VRESP: 0.43000 | VOLD: 0.42500 | VNEW:0.43000 | EMF: 0.119879 | TEMP: 2.935558
09.53% |

16.96% | VMEAS: 0.76613 | VRESP: 0.77000 | VOLD: 0.76500 | VNEW:0.77000 | EMF: 0.119408 | TEMP: 3.023413
17.07% | VMEAS: 0.77084 | VRESP: 0.77500 | VOLD: 0.77000 | VNEW:0.77500 | EMF: 0.119466 | TEMP: 3.024297
17.18% | VMEAS: 0.78091 | VRESP: 0.78000 | VOLD: 0.77500 | VNEW:0.78000 | EMF: 0.119116 | TEMP: 3.033246
17.29% | VMEAS: 0.78596 | VRESP: 0.78500 | VOLD: 0.78000 | VNEW:0.78500 | EMF: 0.119816 | TEMP: 3.051547
17.41% | VMEAS: 0.78668 | VRESP: 0.79000 | VOLD: 0.78500 | VNEW:0.79000 | EMF: 0.119450 | TEMP: 3.053283
17.52% | VMEAS: 0.79600 | VRESP: 0.79500 | VOLD: 0.79000 | VNEW:0.79500 | EMF: 0.119123 | TEMP: 3.047162
17.63% | VMEAS: 0.80103 | VRESP: 0.80000 | VOLD: 0.79500 | VNEW:0.80000 | EMF: 0.119759 | TEMP: 3.039858
s960 | Recovered: |'+00.806080\r'|, buf |+00.|
17.74% | VMEAS: 0.80608 | VRESP: 0.80500 | VOLD: 0.80000 | VNEW:0.80500 | EMF: 0.119381 | TEMP: 3.034746
17.85% | VMEAS: 0.81110 | VRESP: 0.81000 | VOLD: 0.80500 | VNEW:0.81000 | EMF: 0.119048 | TEMP: 3.029034
17.96% |

25.61% | VMEAS: 1.16117 | VRESP: 1.16000 | VOLD: 1.15500 | VNEW:1.16000 | EMF: 0.119066 | TEMP: 3.127149
25.72% | VMEAS: 1.16620 | VRESP: 1.16500 | VOLD: 1.16000 | VNEW:1.16500 | EMF: 0.119701 | TEMP: 3.113855
25.83% | VMEAS: 1.16620 | VRESP: 1.17000 | VOLD: 1.16500 | VNEW:1.17000 | EMF: 0.119299 | TEMP: 3.126802
25.94% | VMEAS: 1.17125 | VRESP: 1.17500 | VOLD: 1.17000 | VNEW:1.17500 | EMF: 0.119937 | TEMP: 3.113402
26.05% | VMEAS: 1.18131 | VRESP: 1.18000 | VOLD: 1.17500 | VNEW:1.18000 | EMF: 0.119482 | TEMP: 3.130192
26.16% | VMEAS: 1.18130 | VRESP: 1.18500 | VOLD: 1.18000 | VNEW:1.18500 | EMF: 0.119066 | TEMP: 3.121197
26.27% | VMEAS: 1.18629 | VRESP: 1.19000 | VOLD: 1.18500 | VNEW:1.19000 | EMF: 0.119669 | TEMP: 3.118278
26.39% | VMEAS: 1.19604 | VRESP: 1.19500 | VOLD: 1.19000 | VNEW:1.19500 | EMF: 0.119247 | TEMP: 3.121631
26.50% | VMEAS: 1.20109 | VRESP: 1.20000 | VOLD: 1.19500 | VNEW:1.20000 | EMF: 0.119274 | TEMP: 3.110514
26.61% | VMEAS: 1.20610 | VRESP: 1.20500 | VOLD: 1.2000

34.26% | VMEAS: 1.55164 | VRESP: 1.55000 | VOLD: 1.54500 | VNEW:1.55000 | EMF: 0.119840 | TEMP: 3.166973
34.37% | VMEAS: 1.55669 | VRESP: 1.55500 | VOLD: 1.55000 | VNEW:1.55500 | EMF: 0.119364 | TEMP: 3.165034
34.48% | VMEAS: 1.56171 | VRESP: 1.56000 | VOLD: 1.55500 | VNEW:1.56000 | EMF: 0.119975 | TEMP: 3.159650
34.59% | VMEAS: 1.56675 | VRESP: 1.56500 | VOLD: 1.56000 | VNEW:1.56500 | EMF: 0.119553 | TEMP: 3.174889
34.70% | VMEAS: 1.57143 | VRESP: 1.57000 | VOLD: 1.56500 | VNEW:1.57000 | EMF: 0.119150 | TEMP: 3.173022
34.81% | VMEAS: 1.57648 | VRESP: 1.57500 | VOLD: 1.57000 | VNEW:1.57500 | EMF: 0.119100 | TEMP: 3.164961
34.92% | VMEAS: 1.58153 | VRESP: 1.58000 | VOLD: 1.57500 | VNEW:1.58000 | EMF: 0.119727 | TEMP: 3.168347
35.03% | VMEAS: 1.58656 | VRESP: 1.58500 | VOLD: 1.58000 | VNEW:1.58500 | EMF: 0.119326 | TEMP: 3.181235
35.14% | VMEAS: 1.58655 | VRESP: 1.59000 | VOLD: 1.58500 | VNEW:1.59000 | EMF: 0.119940 | TEMP: 3.169347
35.25% | VMEAS: 1.59662 | VRESP: 1.59500 | VOLD: 1.5900

...

59.98% | VMEAS: 2.71219 | VRESP: 2.71000 | VOLD: 2.70500 | VNEW:2.71000 | EMF: 0.119720 | TEMP: 3.177072
60.09% | VMEAS: 2.71722 | VRESP: 2.71500 | VOLD: 2.71000 | VNEW:2.71500 | EMF: 0.119170 | TEMP: 3.184728
60.20% | VMEAS: 2.72226 | VRESP: 2.72000 | VOLD: 2.71500 | VNEW:2.72000 | EMF: 0.119711 | TEMP: 3.177293
60.31% | VMEAS: 2.72733 | VRESP: 2.72500 | VOLD: 2.72000 | VNEW:2.72500 | EMF: 0.119205 | TEMP: 3.159656
60.42% | VMEAS: 2.73231 | VRESP: 2.73000 | VOLD: 2.72500 | VNEW:2.73000 | EMF: 0.119744 | TEMP: 3.165619
60.53% | VMEAS: 2.73704 | VRESP: 2.73500 | VOLD: 2.73000 | VNEW:2.73500 | EMF: 0.119169 | TEMP: 3.154034
60.64% | VMEAS: 2.73701 | VRESP: 2.74000 | VOLD: 2.73500 | VNEW:2.74000 | EMF: 0.119084 | TEMP: 3.163288
60.75% | VMEAS: 2.74208 | VRESP: 2.74500 | VOLD: 2.74000 | VNEW:2.74500 | EMF: 0.119644 | TEMP: 3.162817
60.86% | VMEAS: 2.75211 | VRESP: 2.75000 | VOLD: 2.74500 | VNEW:2.75000 | EMF: 0.119141 | TEMP: 3.165141
60.98% | VMEAS: 2.75717 | VRESP: 2.75500 | VOLD: 2.7500

68.51% | VMEAS: 3.09269 | VRESP: 3.09500 | VOLD: 3.09000 | VNEW:3.09500 | EMF: 0.119345 | TEMP: 3.143427
68.63% | VMEAS: 3.09752 | VRESP: 3.10000 | VOLD: 3.09500 | VNEW:3.10000 | EMF: 0.119765 | TEMP: 3.135503
68.74% | VMEAS: 3.10751 | VRESP: 3.10500 | VOLD: 3.10000 | VNEW:3.10500 | EMF: 0.119208 | TEMP: 3.157279
68.85% | VMEAS: 3.11224 | VRESP: 3.11000 | VOLD: 3.10500 | VNEW:3.11000 | EMF: 0.119724 | TEMP: 3.140443
68.96% | VMEAS: 3.11726 | VRESP: 3.11500 | VOLD: 3.11000 | VNEW:3.11500 | EMF: 0.119589 | TEMP: 3.150322
69.07% | VMEAS: 3.12230 | VRESP: 3.12000 | VOLD: 3.11500 | VNEW:3.12000 | EMF: 0.118998 | TEMP: 3.141377
69.18% | VMEAS: 3.12735 | VRESP: 3.12500 | VOLD: 3.12000 | VNEW:3.12500 | EMF: 0.119511 | TEMP: 3.139691
69.29% | VMEAS: 3.13237 | VRESP: 3.13000 | VOLD: 3.12500 | VNEW:3.13000 | EMF: 0.118979 | TEMP: 3.141375
69.40% | VMEAS: 3.13741 | VRESP: 3.13500 | VOLD: 3.13000 | VNEW:3.13500 | EMF: 0.119502 | TEMP: 3.148026
69.51% | VMEAS: 3.14243 | VRESP: 3.14000 | VOLD: 3.1350

77.16% | VMEAS: 3.48751 | VRESP: 3.48500 | VOLD: 3.48000 | VNEW:3.48500 | EMF: 0.119030 | TEMP: 3.108589
77.27% | VMEAS: 3.49256 | VRESP: 3.49000 | VOLD: 3.48500 | VNEW:3.49000 | EMF: 0.119526 | TEMP: 3.115460
77.38% | VMEAS: 3.49760 | VRESP: 3.49500 | VOLD: 3.49000 | VNEW:3.49500 | EMF: 0.119974 | TEMP: 3.116199
77.49% | VMEAS: 3.49760 | VRESP: 3.50000 | VOLD: 3.49500 | VNEW:3.50000 | EMF: 0.119370 | TEMP: 3.105103
77.61% | VMEAS: 3.50259 | VRESP: 3.50500 | VOLD: 3.50000 | VNEW:3.50500 | EMF: 0.119776 | TEMP: 3.119963
77.72% | VMEAS: 3.51267 | VRESP: 3.51000 | VOLD: 3.50500 | VNEW:3.51000 | EMF: 0.119188 | TEMP: 3.128290
77.83% | VMEAS: 3.51771 | VRESP: 3.51500 | VOLD: 3.51000 | VNEW:3.51500 | EMF: 0.119666 | TEMP: 3.126923
77.94% | VMEAS: 3.51771 | VRESP: 3.52000 | VOLD: 3.51500 | VNEW:3.52000 | EMF: 0.119043 | TEMP: 3.122441
78.05% | VMEAS: 3.52275 | VRESP: 3.52500 | VOLD: 3.52000 | VNEW:3.52500 | EMF: 0.119524 | TEMP: 3.109411
78.16% | VMEAS: 3.52777 | VRESP: 3.53000 | VOLD: 3.5250

85.81% | VMEAS: 3.87275 | VRESP: 3.87500 | VOLD: 3.87000 | VNEW:3.87500 | EMF: 0.119552 | TEMP: 3.075641
85.92% | VMEAS: 3.87781 | VRESP: 3.88000 | VOLD: 3.87500 | VNEW:3.88000 | EMF: 0.119970 | TEMP: 3.070310
86.03% | VMEAS: 3.88787 | VRESP: 3.88500 | VOLD: 3.88000 | VNEW:3.88500 | EMF: 0.119315 | TEMP: 3.083803
86.14% | VMEAS: 3.89290 | VRESP: 3.89000 | VOLD: 3.88500 | VNEW:3.89000 | EMF: 0.119737 | TEMP: 3.081433
86.25% | VMEAS: 3.89792 | VRESP: 3.89500 | VOLD: 3.89000 | VNEW:3.89500 | EMF: 0.119064 | TEMP: 3.086226
86.36% | VMEAS: 3.90297 | VRESP: 3.90000 | VOLD: 3.89500 | VNEW:3.90000 | EMF: 0.119512 | TEMP: 3.076270
86.47% | VMEAS: 3.90768 | VRESP: 3.90500 | VOLD: 3.90000 | VNEW:3.90500 | EMF: 0.119945 | TEMP: 3.086349
86.59% | VMEAS: 3.91267 | VRESP: 3.91000 | VOLD: 3.90500 | VNEW:3.91000 | EMF: 0.119711 | TEMP: 3.076127
86.70% | VMEAS: 3.91775 | VRESP: 3.91500 | VOLD: 3.91000 | VNEW:3.91500 | EMF: 0.119098 | TEMP: 3.082345
86.81% | VMEAS: 3.92279 | VRESP: 3.92000 | VOLD: 3.9150

94.46% | VMEAS: 4.26326 | VRESP: 4.26500 | VOLD: 4.26000 | VNEW:4.26500 | EMF: 0.119030 | TEMP: 3.054686
94.57% | VMEAS: 4.26829 | VRESP: 4.27000 | VOLD: 4.26500 | VNEW:4.27000 | EMF: 0.119480 | TEMP: 3.056036
94.68% | VMEAS: 4.27335 | VRESP: 4.27500 | VOLD: 4.27000 | VNEW:4.27500 | EMF: 0.119869 | TEMP: 3.064913
94.79% | VMEAS: 4.27804 | VRESP: 4.28000 | VOLD: 4.27500 | VNEW:4.28000 | EMF: 0.119561 | TEMP: 3.058901
94.90% | VMEAS: 4.28307 | VRESP: 4.28500 | VOLD: 4.28000 | VNEW:4.28500 | EMF: 0.119958 | TEMP: 3.057961
95.01% | VMEAS: 4.29316 | VRESP: 4.29000 | VOLD: 4.28500 | VNEW:4.29000 | EMF: 0.119294 | TEMP: 3.064347
95.12% | VMEAS: 4.29814 | VRESP: 4.29500 | VOLD: 4.29000 | VNEW:4.29500 | EMF: 0.119696 | TEMP: 3.048664
95.23% | VMEAS: 4.30321 | VRESP: 4.30000 | VOLD: 4.29500 | VNEW:4.30000 | EMF: 0.118990 | TEMP: 3.042445
95.34% | VMEAS: 4.30786 | VRESP: 4.30500 | VOLD: 4.30000 | VNEW:4.30500 | EMF: 0.119416 | TEMP: 3.037306
95.45% | VMEAS: 4.31285 | VRESP: 4.31000 | VOLD: 4.3050

### Lets plot our ramp!

In [ ]:
%matplotlib notebook
fig,ax1 = plt.subplots()
l1, = ax1.plot(times_emf,np.array(emf_series)*1000,'-r',label='EMF')
ax1.set_ylabel("EMF (mV)")
ax1.set_xlabel("Time (s)")
ax2 = ax1.twinx()
l2, = ax2.plot(times_vmeas,np.array(vmeas_series)/0.5464,'-b',label='IOUT')
l3, = ax2.plot(times_tempFAA,tempFAA_series,'-g',label='FAA')
ax2.set_ylabel("OUTPUT (A) and TEMPERATURE (K)")
plt.legend([l1,l2,l3], [l.get_label() for l in [l1,l2,l3]])
plt.tight_layout()
plt.show()

## 3. Soak
To allow for complete thermalization and spin aligment, field is held constant.
Depending on who you ask, anywhere from 5 minutes to multiple hours is sufficient.
There are strongly diminishing returns, so we only hold for 10 minutes this time.

In [ ]:
# time.sleep(10*60)
# j/k

## 4. DeMag
Now comes the fun part - we first open the heat switch to decouple mK stage from 4K

In [10]:
sd.heatswitch_open() # OPEN HS!!!

Heat Switch opening
4K-GGG LED ON
4K-FAA LED OFF
GGG-FAA LED OFF
Heat Switch almost open - this should be ok and will probably go away at lower currents


And then start a loop similar to what we had before. Physically, salt pill will cool down mK stages as entropy of magnetic moments increases (and thermal one decreases). This is only possible due to quasi-adiabatic, constant entropy nature of this process and so demag must be done slowly.

In [11]:
target = 0.0 #A
maxemf = 0.120 #V
vstep = 0.005 #Max slew rate in V/s
target=target*0.55 #convert to V

In [12]:
# Define some empty lists for logging
emf_series2, vmeas_series2, times_emf2, times_vmeas2 = ([] for i in range(4))
tempFAA_series2, times_tempFAA2 = ([] for i in range(2))

# Get current output twice - if this is somehow wrong, things will go very bad
vout = s960.get_manual_output()
vout2 = s960.get_manual_output()
assert vout == vout2

# Can go either direction
if vout > target: 
    direction = 0 #down
elif vout == target: 
    warnings.warn("Target same as current value")
    direction = 1
else:
    direction = 1 #up
delta = abs(vout - target)

# Flush communications
mf.clear_virtual_queues()
mf.clear_hw_queues()

# Get some pretty bars
barv = make_bar(min(target,vout),max(target,vout),prefix='Measured output: ',postfix='V',style='warning')
baremf = make_bar(0,300,prefix='Back EMF: ',postfix='mV',style='danger')
bart = make_bar(0,3,prefix='FAA: ',postfix='K',style='info')

print("Starting run from {} to {} (dir:{}) at {} with emf limit of {}".format(vout,target,direction,vstep,maxemf))
start = timer()
try:
    # Start streaming EMF channel data
    s970.start_data_stream(SRS.SIM970.CHANNELS.MAGEMF)
    # And also temperatures
    s921.start_data_stream()
    
    while(abs(vout - target) > 0.0):
        loop_start = timer()        
        # Loop until backemf is low enough, but at least once (poor mans do-while loop)
        while True:            
            gotevent = s970.await_next_event(tm=1.0,clear_before=True,clear_after=True)
            if not gotevent:
                raise RuntimeError("Did not get voltage in time - aborting")   
            emf = abs(s970.lastvalues[SRS.SIM970.CHANNELS.MAGEMF])
            emf_series2.append(emf)
            times_emf2.append(timer()-start)
            update_bar(baremf,emf*1000)            
            if abs(emf) < abs(maxemf): break
            
        # Set new manual voltage
        if (direction):
            if (vout + vstep < target): vnew = vout + vstep               
            else: vnew = target                   
        else:
            if (vout - vstep > target): vnew = vout - vstep                
            else: vnew = target                 
        s960.set_manual_output(vnew)
        time.sleep(0.2)
        
        # Check that new manual voltage is as intended (setting and actual output)
        # Sometimes SRS goes crazy, so we try two times
        attempts = 0
        while attempts < 2:
            try:
                vout_chk = s960.get_manual_output()
                time.sleep(0.2)
                vmeas_chk = s960.get_output_voltage()
                time.sleep(0.2)  
                break
            except Exception as e: 
                warnings.warn(e)
                attempts += 1
        if attempts >=2: raise RuntimeError("Could not get s960 settings - ABORTING")      
                
        # Grab new temperature reading        
        if s921.await_next_event(tm=1.1,clear_before=False,clear_after=True):
            temp = s921.lastvalues[1]
            tempFAA_series2.append(temp)
            times_tempFAA2.append(s921.lastdatatimes[1]-start)
            update_bar(bart,temp)
        else:
            print("Got not temp data...weird...")
            temp = 0
            
        print("{:05.2f}% | VMEAS: {:.5f} | VRESP: {:.5f} | VOLD: {:.5f} | VNEW:{:.5f} | EMF: {:.6f} | TEMP: {:.6f}".format(\
            100-abs(vout - target)*100/delta,vmeas_chk,vout_chk,vout,vnew,emf,temp))
        vout = vnew; vmeas_series2.append(vmeas_chk); times_vmeas2.append(timer()-start)
        update_bar(barv,vout_chk)
        
        # Error checking
        check_for_exceptions()
        assert abs(vout_chk - vnew) < 0.000001 #rounding of floats a problem 
        assert abs(vmeas_chk - vnew) < 0.02        
        
        # Wait remainder of 1 second
        while(timer()-loop_start < 1.0): time.sleep(0.05)  
    print("DONE - took {:2f} minutes".format((timer()-start)/60))
finally:
    # Stop data and clean up
    s970.stop_data_stream()
    s921.stop_data_stream()
    time.sleep(0.2)
    mf.clear_virtual_queues()

Starting run from 4.51 to 0.0 (dir:0) at 0.005 with emf limit of 0.12
00.00% | VMEAS: 4.50824 | VRESP: 4.50500 | VOLD: 4.51000 | VNEW:4.50500 | EMF: 0.000240 | TEMP: 2.895889
00.11% | VMEAS: 4.50824 | VRESP: 4.50000 | VOLD: 4.50500 | VNEW:4.50000 | EMF: 0.009557 | TEMP: 2.886601
00.22% | VMEAS: 4.50283 | VRESP: 4.49500 | VOLD: 4.50000 | VNEW:4.49500 | EMF: 0.018906 | TEMP: 2.895461
00.33% | VMEAS: 4.49314 | VRESP: 4.49000 | VOLD: 4.49500 | VNEW:4.49000 | EMF: 0.028206 | TEMP: 2.886211
00.44% | VMEAS: 4.49311 | VRESP: 4.48500 | VOLD: 4.49000 | VNEW:4.48500 | EMF: 0.036846 | TEMP: 2.893876
00.55% | VMEAS: 4.48343 | VRESP: 4.48000 | VOLD: 4.48500 | VNEW:4.48000 | EMF: 0.044425 | TEMP: 2.889437
00.67% | VMEAS: 4.47836 | VRESP: 4.47500 | VOLD: 4.48000 | VNEW:4.47500 | EMF: 0.052380 | TEMP: 2.887031
00.78% | VMEAS: 4.47333 | VRESP: 4.47000 | VOLD: 4.47500 | VNEW:4.47000 | EMF: 0.060071 | TEMP: 2.895957
00.89% | VMEAS: 4.47333 | VRESP: 4.46500 | VOLD: 4.47000 | VNEW:4.46500 | EMF: 0.067424 | 

08.54% | VMEAS: 4.12280 | VRESP: 4.12000 | VOLD: 4.12500 | VNEW:4.12000 | EMF: 0.119432 | TEMP: 2.732031
08.65% | VMEAS: 4.11777 | VRESP: 4.11500 | VOLD: 4.12000 | VNEW:4.11500 | EMF: 0.119999 | TEMP: 2.729672
08.76% | VMEAS: 4.11308 | VRESP: 4.11000 | VOLD: 4.11500 | VNEW:4.11000 | EMF: 0.119209 | TEMP: 2.722384
08.87% | VMEAS: 4.10806 | VRESP: 4.10500 | VOLD: 4.11000 | VNEW:4.10500 | EMF: 0.119944 | TEMP: 2.709108
08.98% | VMEAS: 4.10300 | VRESP: 4.10000 | VOLD: 4.10500 | VNEW:4.10000 | EMF: 0.119153 | TEMP: 2.720571
09.09% | VMEAS: 4.09801 | VRESP: 4.09500 | VOLD: 4.10000 | VNEW:4.09500 | EMF: 0.119419 | TEMP: 2.709503
09.20% | VMEAS: 4.09297 | VRESP: 4.09000 | VOLD: 4.09500 | VNEW:4.09000 | EMF: 0.119656 | TEMP: 2.710698
09.31% | VMEAS: 4.08794 | VRESP: 4.08500 | VOLD: 4.09000 | VNEW:4.08500 | EMF: 0.119921 | TEMP: 2.723104
09.42% | VMEAS: 4.08287 | VRESP: 4.08000 | VOLD: 4.08500 | VNEW:4.08000 | EMF: 0.119142 | TEMP: 2.726506
09.53% | VMEAS: 4.07785 | VRESP: 4.07500 | VOLD: 4.0800

17.07% | VMEAS: 3.73787 | VRESP: 3.73500 | VOLD: 3.74000 | VNEW:3.73500 | EMF: 0.119903 | TEMP: 2.541608
s960 | Recovered: |'+3.730\r'|, buf |+3.7|
17.18% | VMEAS: 3.73730 | VRESP: 3.73000 | VOLD: 3.73500 | VNEW:3.73000 | EMF: 0.119115 | TEMP: 2.536914
17.29% | VMEAS: 3.73223 | VRESP: 3.72500 | VOLD: 3.73000 | VNEW:3.72500 | EMF: 0.119419 | TEMP: 2.523334
17.41% | VMEAS: 3.72783 | VRESP: 3.72000 | VOLD: 3.72500 | VNEW:3.72000 | EMF: 0.119720 | TEMP: 2.518044
17.52% | VMEAS: 3.71775 | VRESP: 3.71500 | VOLD: 3.72000 | VNEW:3.71500 | EMF: 0.118976 | TEMP: 2.528091
17.63% | VMEAS: 3.71273 | VRESP: 3.71000 | VOLD: 3.71500 | VNEW:3.71000 | EMF: 0.119336 | TEMP: 2.525376
17.74% | VMEAS: 3.70768 | VRESP: 3.70500 | VOLD: 3.71000 | VNEW:3.70500 | EMF: 0.119587 | TEMP: 2.530952
17.85% | VMEAS: 3.70264 | VRESP: 3.70000 | VOLD: 3.70500 | VNEW:3.70000 | EMF: 0.119897 | TEMP: 2.516979
17.96% | VMEAS: 3.69761 | VRESP: 3.69500 | VOLD: 3.70000 | VNEW:3.69500 | EMF: 0.119127 | TEMP: 2.504473
18.07% | VME

25.61% | VMEAS: 3.35758 | VRESP: 3.35000 | VOLD: 3.35500 | VNEW:3.35000 | EMF: 0.119138 | TEMP: 2.322269
25.72% | VMEAS: 3.35253 | VRESP: 3.34500 | VOLD: 3.35000 | VNEW:3.34500 | EMF: 0.119476 | TEMP: 2.321287
25.83% | VMEAS: 3.34750 | VRESP: 3.34000 | VOLD: 3.34500 | VNEW:3.34000 | EMF: 0.119776 | TEMP: 2.325283
25.94% | VMEAS: 3.33744 | VRESP: 3.33500 | VOLD: 3.34000 | VNEW:3.33500 | EMF: 0.119021 | TEMP: 2.315603
26.05% | VMEAS: 3.33241 | VRESP: 3.33000 | VOLD: 3.33500 | VNEW:3.33000 | EMF: 0.119410 | TEMP: 2.299780
26.16% | VMEAS: 3.32738 | VRESP: 3.32500 | VOLD: 3.33000 | VNEW:3.32500 | EMF: 0.119723 | TEMP: 2.303211
26.27% | VMEAS: 3.32235 | VRESP: 3.32000 | VOLD: 3.32500 | VNEW:3.32000 | EMF: 0.119971 | TEMP: 2.305670
26.39% | VMEAS: 3.31848 | VRESP: 3.31500 | VOLD: 3.32000 | VNEW:3.31500 | EMF: 0.119198 | TEMP: 2.297691
26.50% | VMEAS: 3.31261 | VRESP: 3.31000 | VOLD: 3.31500 | VNEW:3.31000 | EMF: 0.119961 | TEMP: 2.297241
26.61% | VMEAS: 3.30901 | VRESP: 3.30500 | VOLD: 3.3100

34.15% | VMEAS: 2.96727 | VRESP: 2.96500 | VOLD: 2.97000 | VNEW:2.96500 | EMF: 0.119114 | TEMP: 2.086607
34.26% | VMEAS: 2.96225 | VRESP: 2.96000 | VOLD: 2.96500 | VNEW:2.96000 | EMF: 0.119471 | TEMP: 2.089252
34.37% | VMEAS: 2.95720 | VRESP: 2.95500 | VOLD: 2.96000 | VNEW:2.95500 | EMF: 0.119824 | TEMP: 2.092031
34.48% | VMEAS: 2.95447 | VRESP: 2.95000 | VOLD: 2.95500 | VNEW:2.95000 | EMF: 0.119082 | TEMP: 2.081658
34.59% | VMEAS: 2.94745 | VRESP: 2.94500 | VOLD: 2.95000 | VNEW:2.94500 | EMF: 0.119462 | TEMP: 2.089113
34.70% | VMEAS: 2.94244 | VRESP: 2.94000 | VOLD: 2.94500 | VNEW:2.94000 | EMF: 0.119169 | TEMP: 2.087752
34.81% | VMEAS: 2.93780 | VRESP: 2.93500 | VOLD: 2.94000 | VNEW:2.93500 | EMF: 0.119478 | TEMP: 2.096542
34.92% | VMEAS: 2.93274 | VRESP: 2.93000 | VOLD: 2.93500 | VNEW:2.93000 | EMF: 0.119812 | TEMP: 2.099650
35.03% | VMEAS: 2.92733 | VRESP: 2.92500 | VOLD: 2.93000 | VNEW:2.92500 | EMF: 0.119108 | TEMP: 2.086717
35.14% | VMEAS: 2.92231 | VRESP: 2.92000 | VOLD: 2.9250

...

60.09% | VMEAS: 1.80152 | VRESP: 1.79500 | VOLD: 1.80000 | VNEW:1.79500 | EMF: 0.119030 | TEMP: 1.317104
60.20% | VMEAS: 1.79653 | VRESP: 1.79000 | VOLD: 1.79500 | VNEW:1.79000 | EMF: 0.119585 | TEMP: 1.313886
60.31% | VMEAS: 1.78645 | VRESP: 1.78500 | VOLD: 1.79000 | VNEW:1.78500 | EMF: 0.119074 | TEMP: 1.313408
60.42% | VMEAS: 1.78144 | VRESP: 1.78000 | VOLD: 1.78500 | VNEW:1.78000 | EMF: 0.119598 | TEMP: 1.317162
60.53% | VMEAS: 1.78125 | VRESP: 1.77500 | VOLD: 1.78000 | VNEW:1.77500 | EMF: 0.119039 | TEMP: 1.313344
60.64% | VMEAS: 1.77167 | VRESP: 1.77000 | VOLD: 1.77500 | VNEW:1.77000 | EMF: 0.119977 | TEMP: 1.313990
60.75% | VMEAS: 1.77167 | VRESP: 1.76500 | VOLD: 1.77000 | VNEW:1.76500 | EMF: 0.119448 | TEMP: 1.306746
60.86% | VMEAS: 1.76660 | VRESP: 1.76000 | VOLD: 1.76500 | VNEW:1.76000 | EMF: 0.119962 | TEMP: 1.303208
60.98% | VMEAS: 1.75655 | VRESP: 1.75500 | VOLD: 1.76000 | VNEW:1.75500 | EMF: 0.119426 | TEMP: 1.302828
61.09% | VMEAS: 1.75150 | VRESP: 1.75000 | VOLD: 1.7550

68.74% | VMEAS: 1.40645 | VRESP: 1.40500 | VOLD: 1.41000 | VNEW:1.40500 | EMF: 0.119541 | TEMP: 1.041539
68.85% | VMEAS: 1.40142 | VRESP: 1.40000 | VOLD: 1.40500 | VNEW:1.40000 | EMF: 0.119590 | TEMP: 1.039239
68.96% | VMEAS: 1.39643 | VRESP: 1.39500 | VOLD: 1.40000 | VNEW:1.39500 | EMF: 0.119244 | TEMP: 1.039331
69.07% | VMEAS: 1.39532 | VRESP: 1.39000 | VOLD: 1.39500 | VNEW:1.39000 | EMF: 0.119863 | TEMP: 1.035690
69.18% | VMEAS: 1.38636 | VRESP: 1.38500 | VOLD: 1.39000 | VNEW:1.38500 | EMF: 0.119472 | TEMP: 1.031646
69.29% | VMEAS: 1.38130 | VRESP: 1.38000 | VOLD: 1.38500 | VNEW:1.38000 | EMF: 0.119133 | TEMP: 1.032401
69.40% | VMEAS: 1.37624 | VRESP: 1.37500 | VOLD: 1.38000 | VNEW:1.37500 | EMF: 0.119833 | TEMP: 1.028140
69.51% | VMEAS: 1.37106 | VRESP: 1.37000 | VOLD: 1.37500 | VNEW:1.37000 | EMF: 0.119483 | TEMP: 1.023544
69.62% | VMEAS: 1.36604 | VRESP: 1.36500 | VOLD: 1.37000 | VNEW:1.36500 | EMF: 0.119426 | TEMP: 1.019879
69.73% | VMEAS: 1.36603 | VRESP: 1.36000 | VOLD: 1.3650

77.38% | VMEAS: 1.01597 | VRESP: 1.01500 | VOLD: 1.02000 | VNEW:1.01500 | EMF: 0.119990 | TEMP: 0.765752
77.49% | VMEAS: 1.01139 | VRESP: 1.01000 | VOLD: 1.01500 | VNEW:1.01000 | EMF: 0.119688 | TEMP: 0.759915
77.61% | VMEAS: 1.00588 | VRESP: 1.00500 | VOLD: 1.01000 | VNEW:1.00500 | EMF: 0.119411 | TEMP: 0.756712
77.72% | VMEAS: 1.00085 | VRESP: 1.00000 | VOLD: 1.00500 | VNEW:1.00000 | EMF: 0.119092 | TEMP: 0.753779
77.83% | VMEAS: 0.99581 | VRESP: 0.99500 | VOLD: 1.00000 | VNEW:0.99500 | EMF: 0.119837 | TEMP: 0.750675
77.94% | VMEAS: 0.99078 | VRESP: 0.99000 | VOLD: 0.99500 | VNEW:0.99000 | EMF: 0.119537 | TEMP: 0.750814
78.05% | VMEAS: 0.98574 | VRESP: 0.98500 | VOLD: 0.99000 | VNEW:0.98500 | EMF: 0.119239 | TEMP: 0.741124
78.16% | VMEAS: 0.98105 | VRESP: 0.98000 | VOLD: 0.98500 | VNEW:0.98000 | EMF: 0.119988 | TEMP: 0.739124
78.27% | VMEAS: 0.98103 | VRESP: 0.97500 | VOLD: 0.98000 | VNEW:0.97500 | EMF: 0.119046 | TEMP: 0.735798
78.38% | VMEAS: 0.97094 | VRESP: 0.97000 | VOLD: 0.9750

86.14% | VMEAS: 0.62050 | VRESP: 0.62000 | VOLD: 0.62500 | VNEW:0.62000 | EMF: 0.119585 | TEMP: 0.494257
86.25% | VMEAS: 0.61550 | VRESP: 0.61500 | VOLD: 0.62000 | VNEW:0.61500 | EMF: 0.119214 | TEMP: 0.492134
86.36% | VMEAS: 0.61080 | VRESP: 0.61000 | VOLD: 0.61500 | VNEW:0.61000 | EMF: 0.119921 | TEMP: 0.488017
86.47% | VMEAS: 0.60573 | VRESP: 0.60500 | VOLD: 0.61000 | VNEW:0.60500 | EMF: 0.119984 | TEMP: 0.486321
86.59% | VMEAS: 0.60070 | VRESP: 0.60000 | VOLD: 0.60500 | VNEW:0.60000 | EMF: 0.119605 | TEMP: 0.481822
86.70% | VMEAS: 0.59565 | VRESP: 0.59500 | VOLD: 0.60000 | VNEW:0.59500 | EMF: 0.119272 | TEMP: 0.480200
86.81% | VMEAS: 0.59060 | VRESP: 0.59000 | VOLD: 0.59500 | VNEW:0.59000 | EMF: 0.119971 | TEMP: 0.473687
86.92% | VMEAS: 0.59062 | VRESP: 0.58500 | VOLD: 0.59000 | VNEW:0.58500 | EMF: 0.119628 | TEMP: 0.470397
87.03% | VMEAS: 0.58056 | VRESP: 0.58000 | VOLD: 0.58500 | VNEW:0.58000 | EMF: 0.119283 | TEMP: 0.468009
87.14% | VMEAS: 0.57553 | VRESP: 0.57500 | VOLD: 0.5800

94.90% | VMEAS: 0.23035 | VRESP: 0.22500 | VOLD: 0.23000 | VNEW:0.22500 | EMF: 0.119563 | TEMP: 0.234105
95.01% | VMEAS: 0.22045 | VRESP: 0.22000 | VOLD: 0.22500 | VNEW:0.22000 | EMF: 0.119254 | TEMP: 0.230926
95.12% | VMEAS: 0.21538 | VRESP: 0.21500 | VOLD: 0.22000 | VNEW:0.21500 | EMF: 0.119932 | TEMP: 0.228655
95.23% | VMEAS: 0.21093 | VRESP: 0.21000 | VOLD: 0.21500 | VNEW:0.21000 | EMF: 0.119583 | TEMP: 0.224928
95.34% | VMEAS: 0.20532 | VRESP: 0.20500 | VOLD: 0.21000 | VNEW:0.20500 | EMF: 0.119270 | TEMP: 0.221700
95.45% | VMEAS: 0.20028 | VRESP: 0.20000 | VOLD: 0.20500 | VNEW:0.20000 | EMF: 0.119995 | TEMP: 0.219840
95.57% | VMEAS: 0.19525 | VRESP: 0.19500 | VOLD: 0.20000 | VNEW:0.19500 | EMF: 0.119657 | TEMP: 0.215604
95.68% | VMEAS: 0.19521 | VRESP: 0.19000 | VOLD: 0.19500 | VNEW:0.19000 | EMF: 0.119319 | TEMP: 0.212429
95.79% | VMEAS: 0.19024 | VRESP: 0.18500 | VOLD: 0.19000 | VNEW:0.18500 | EMF: 0.119971 | TEMP: 0.209759
95.90% | VMEAS: 0.18046 | VRESP: 0.18000 | VOLD: 0.1850

### Lets plot our ramp!

In [ ]:
%matplotlib notebook
fig,ax1 = plt.subplots()
l1, = ax1.plot(times_emf2,np.array(emf_series2)*1000,'-r',label='EMF')
ax1.set_ylabel("EMF (mV)")
ax1.set_xlabel("Time (s)")
ax2 = ax1.twinx()
l2, = ax2.plot(times_vmeas2,np.array(vmeas_series2)/0.5464,'-b',label='IOUT')
l3, = ax2.plot(times_tempFAA2,tempFAA_series2,'-g',label='FAA')
ax2.set_ylabel("OUTPUT (A) and TEMPERATURE (K)")
plt.legend([l1,l2,l3], [l.get_label() for l in [l1,l2,l3]])
plt.tight_layout()
plt.show()

#### You are done with cooldown part - proceed to other data taking

## Debug things - not too useful for ADR learning, but you can look if you want to...

In [ ]:
mf.sim900ser.setBreak(0.5)

In [ ]:
mf.reset()

In [ ]:
mf.reset_serial_module(6)

In [ ]:
s921.start_data_stream()

In [ ]:
s921.await_next_event(tm=2.5)

In [ ]:
s921.stop_data_stream()